In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
files.upload()

Saving 2001_districts_quant.csv to 2001_districts_quant.csv
Saving 2003_districts_quant.csv to 2003_districts_quant.csv
Saving 2005_districts_quant.csv to 2005_districts_quant.csv
Saving 2007_districts_quant.csv to 2007_districts_quant.csv
Saving 2009_districts_quant.csv to 2009_districts_quant.csv
Saving 2011_districts_quant.csv to 2011_districts_quant.csv
Saving District - Ground Truth - 2011&2001.csv to District - Ground Truth - 2011&2001.csv
Saving FEMP&LIT.csv to FEMP&LIT.csv


{'2001_districts_quant.csv': b',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120\n1,1,4136,2461,844,212,132,95,125,152,217,1406,3118,1458,1095,847,694,479,240,238,379,1232,2234,1511,967,873,893,789,612,395,511,995,4899,1582,912,579,334,178,129,161,166,840,4462,1103,673,728,808,710,628,481,180,7,3739,4052,868,615,259,138,63,36,10,0,3808,3769,1196,493,260,142,65,36,11,0,3200,1400,884,644,742,812,831,853,396,18,4436,1229,860,621,512,364,222,189,281,1066,677,1078,1297,1305,1105,912,872,1010,1350,174,3185,1323,780,618,629,746,873,859,551,216,364,680,771,695,571,559,564,596,892,4088\n2,2,928,1474,1416,462,268,124,66,29,53,1384,645,373,634,862,841,609,420,323,137,1360,501,3

In [20]:
df_2001 = pd.read_csv("2001_districts_quant.csv")
df_2003 = pd.read_csv("2003_districts_quant.csv")
df_2005 = pd.read_csv("2005_districts_quant.csv")
df_2007 = pd.read_csv("2007_districts_quant.csv")
df_2009 = pd.read_csv("2009_districts_quant.csv")
df_2011 = pd.read_csv("2011_districts_quant.csv")
gt = pd.read_csv("District - Ground Truth - 2011&2001.csv")
df_features = pd.read_csv("FEMP&LIT.csv")

In [21]:
df_2001.shape,df_2003.shape,df_2005.shape,df_2007.shape,df_2009.shape,df_2011.shape,gt.shape

((640, 122),
 (640, 122),
 (640, 122),
 (640, 122),
 (640, 122),
 (640, 122),
 (593, 17))

In [22]:
df_2003.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '111', '112', '113', '114', '115', '116', '117', '118', '119', '120'],
      dtype='object', length=122)

In [23]:
df_2001["census_code"] = df_2001["0"]
df_2003["census_code"] = df_2003["0"]
df_2005["census_code"] = df_2005["0"]
df_2007["census_code"] = df_2007["0"]
df_2009["census_code"] = df_2009["0"]
df_2011["census_code"] = df_2011["censuscode"]

In [24]:
gt = gt.merge(df_features,on = "census_code")
df2001 = gt.merge(df_2001,on= "census_code")
df2003 = gt.merge(df_2003,on= "census_code")
df2005 = gt.merge(df_2005,on= "census_code")
df2007 = gt.merge(df_2007,on= "census_code")
df2009= gt.merge(df_2009,on= "census_code")
df2011 = gt.merge(df_2011,on= "census_code")

In [25]:
df2001.shape,df2003.shape,df2005.shape,df2007.shape,df2009.shape,df2011.shape

((593, 143), (593, 143), (593, 143), (593, 143), (593, 143), (593, 143))

In [26]:
cols = [str(i) for i in range(1,121)]

In [55]:
col_sec =["2003","2005","2007","2009","2001"]
features = ["ASSET","BF","CHH","FC","MSL","MSW","LIT","FEMP"]
for feature in features :
  clg = XGBClassifier()
  final_train = df2001
  clg.fit(df2001[cols],df2001[feature+"_2001"])
  final_train["2001"] = clg.predict(df2001[cols])
  final_train["2003"] = clg.predict(df2003[cols])
  final_train["2005"] = clg.predict(df2005[cols])
  final_train["2007"] = clg.predict(df2007[cols])
  final_train["2009"] = clg.predict(df2009[cols])
  X_train, X_test, y_train, y_test = train_test_split(final_train[col_sec], final_train[feature+"_2011"], test_size=0.30, random_state=42)
  clf =XGBClassifier()
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(feature, " => ",f1_score(y_test,y_pred, average='weighted'))


ASSET  =>  0.49352563027558827
BF  =>  0.753308304710174
CHH  =>  0.6235601609319944
FC  =>  0.8469742606609272
MSL  =>  0.7848822816083567
MSW  =>  0.855198890536359
LIT  =>  0.5220596878190891
FEMP  =>  0.8827902780557382


In [58]:
col_sec =["2003","2005","2007","2009","2001"]
features = ["ASSET","BF","CHH","FC","MSL","MSW"]
for feature in features :
  clg = XGBClassifier()
  final_train = df2001
  clg.fit(df2001[cols],df2001[feature+"_2001"])
  final_train["2001"] = clg.predict(df2001[cols])
  final_train["2003"] = clg.predict(df2003[cols])
  final_train["2005"] = clg.predict(df2005[cols])
  final_train["2007"] = clg.predict(df2007[cols])
  final_train["2009"] = clg.predict(df2009[cols])
  X_train, X_test, y_train, y_test = train_test_split(final_train[col_sec+[feature+"_2001","LIT_2001","FEMP_2001"]], final_train[feature+"_2011"], test_size=0.30, random_state=42)
  clf =XGBClassifier()
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(feature, " => ",f1_score(y_test,y_pred, average='weighted'))


ASSET  =>  0.7052183964242924
BF  =>  0.8288101387287748
CHH  =>  0.7258266738769054
FC  =>  0.8639884642283329
MSL  =>  0.8225722612209143
MSW  =>  0.8649822450404653


In [47]:
features = ["ASSET","BF","CHH","FC","MSL","MSW"]
col_sec =["2003","2005","2007","2009","2011"]
for feature in features :
  clg = XGBClassifier()
  final_train = df2011
  clg.fit(df2011[cols],df2011[feature+"_2011"])
  final_train["2011"] = clg.predict(df2011[cols])
  final_train["2003"] = clg.predict(df2003[cols])
  final_train["2005"] = clg.predict(df2005[cols])
  final_train["2007"] = clg.predict(df2007[cols])
  final_train["2009"] = clg.predict(df2009[cols])
  X_train, X_test, y_train, y_test = train_test_split(final_train[col_sec], final_train[feature+"_2001"], test_size=0.30, random_state=42)
  clf =XGBClassifier()
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(feature, " => ",f1_score(y_test,y_pred, average='weighted'))

ASSET  =>  0.7742439881617392
BF  =>  0.6843224148116244
CHH  =>  0.6573949648366677
FC  =>  0.8168907263056208
MSL  =>  0.6974636070002161
MSW  =>  0.7838480917628307


In [48]:
features = ["ASSET","BF","CHH","FC","MSL","MSW"]
col_sec =["2003","2005","2007","2009","2011"]
for feature in features :
  clg = XGBClassifier()
  final_train = df2011
  clg.fit(df2011[cols],df2011[feature+"_2011"])
  final_train["2011"] = clg.predict(df2011[cols])
  final_train["2003"] = clg.predict(df2003[cols])
  final_train["2005"] = clg.predict(df2005[cols])
  final_train["2007"] = clg.predict(df2007[cols])
  final_train["2009"] = clg.predict(df2009[cols])
  X_train, X_test, y_train, y_test = train_test_split(final_train[col_sec+[feature+"_2011","LIT_2011","FEMP_2011"]], final_train[feature+"_2001"], test_size=0.30, random_state=42)
  clf =XGBClassifier()
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(feature, " => ",f1_score(y_test,y_pred, average='weighted'))

ASSET  =>  0.8002432097017349
BF  =>  0.7863685465143486
CHH  =>  0.6734806456141172
FC  =>  0.9084481848282532
MSL  =>  0.7266428283529143
MSW  =>  0.7892235625381692


In [38]:
from google.colab import files
files.upload()

Saving 2013_districts_quant.csv to 2013_districts_quant.csv
Saving 2015_districts_quant.csv to 2015_districts_quant.csv
Saving 2017_districts_quant.csv to 2017_districts_quant.csv
Saving 2019_districts_quant.csv to 2019_districts_quant.csv


{'2013_districts_quant.csv': b',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120\n0,0,27712,35153,19791,13756,11204,11746,12779,13754,22247,254756,19781,28106,19754,12765,12830,13098,13269,18539,44364,240392,11831,27814,22165,15265,14043,14611,17419,24521,55874,219355,121009,31398,16639,13008,11210,11853,12353,14565,17053,173810,208509,50215,29231,24668,21190,19140,19615,19983,17848,12499,320167,55079,8362,10022,5727,6113,4801,4511,3917,4198,321235,51796,12790,7634,6240,4831,5394,4997,3842,4139,107337,62309,45103,32941,27921,24558,25378,29431,38119,29801,69677,24430,17410,12949,11692,10759,13835,17317,36587,208190,300647,46585,14686,10296,9255,9724,11426,11911,6866,1

In [51]:
df_2013 = pd.read_csv("2013_districts_quant.csv")
df_2015 = pd.read_csv("2015_districts_quant.csv")
df_2017 = pd.read_csv("2017_districts_quant.csv")
df_2019 = pd.read_csv("2019_districts_quant.csv")

In [52]:
df_2013["census_code"] = df_2013["0"]
df_2015["census_code"] = df_2015["0"]
df_2017["census_code"] = df_2017["0"]
df_2019["census_code"] = df_2019["0"]

In [53]:
df2013 = gt.merge(df_2013,on= "census_code")
df2015 = gt.merge(df_2015,on= "census_code")
df2017 = gt.merge(df_2017,on= "census_code")
df2019= gt.merge(df_2019,on= "census_code")

In [59]:
pred_2019 = pd.DataFrame()
pred_2019["census_code"] = df2011["census_code"]

In [61]:
col_sec =["2003","2005","2007","2009","2001"]
features = ["ASSET","BF","CHH","FC","MSL","MSW"]
for feature in features :
  clg = XGBClassifier()
  final_train = df2001
  clg.fit(df2001[cols],df2001[feature+"_2001"])
  final_train["2001"] = clg.predict(df2001[cols])
  final_train["2003"] = clg.predict(df2003[cols])
  final_train["2005"] = clg.predict(df2005[cols])
  final_train["2007"] = clg.predict(df2007[cols])
  final_train["2009"] = clg.predict(df2009[cols])
  X_train, X_test, y_train, y_test = train_test_split(final_train[col_sec+[feature+"_2001","LIT_2001","FEMP_2001"]], final_train[feature+"_2011"], test_size=0.30, random_state=42)
  clf =XGBClassifier()
  clf.fit(X_train,y_train)
  clg2 = XGBClassifier()
  predictor = df2011
  clg2.fit(df2011[cols],df2011[feature+"_2011"])
  predictor["2001"] = clg2.predict(df2011[cols])
  predictor["2003"] = clg2.predict(df2013[cols])
  predictor["2005"] = clg2.predict(df2015[cols])
  predictor["2007"] = clg2.predict(df2017[cols])
  predictor["2009"] = clg2.predict(df2009[cols])
  predictor[feature+"_2001"] = df2011[feature+"_2011"]
  predictor["LIT_2001"] = df2011["LIT_2011"]
  predictor["FEMP_2001"] = df2011["FEMP_2011"]
  pred_2019[feature] = clf.predict(predictor[col_sec+[feature+"_2001","LIT_2001","FEMP_2001"]])


In [65]:
pred_2019["AQI"] = pred_2019["ASSET"] + pred_2019["BF"]+pred_2019["CHH"]+pred_2019["FC"]+pred_2019["MSL"]+pred_2019["MSW"]

In [66]:
pred_2019.head()

,census_code,ASSET,BF,CHH,FC,MSL,MSW,AQI
0,1,2,1,2,1,3,3,12
1,2,3,1,2,1,3,3,13
2,3,3,3,3,3,3,1,16
3,4,3,3,3,1,3,3,16
4,5,3,1,2,1,3,3,13


In [68]:
pred_2019.to_csv("Predictions_2019.csv")